In [1]:
SECRET_KEY = 'LyyFU_r17F6s1i0ajYI2dxoSi2dOtw'
CLIENT_ID = 'SMi4R1E-3TeMoXqqwPNwUg'
USER_NAME = 'Weary-Tooth7440'

In [2]:
%pip install praw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [8]:
import praw

# Initialize Reddit instance
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_KEY,
                     user_agent=USER_NAME)

In [9]:
# Subreddit to scrape
subreddit = reddit.subreddit('stocks')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

Display Name: stocks
Title: Stocks - Investing and trading for all
Description: Almost any post related to stocks is welcome; please read the rules below:

[**If you're new here**](https://www.reddit.com/r/stocks/comments/4x1419/if_youre_new_here_read_this_post_first/)

##Resources

* [Wiki for new investors](https://www.reddit.com/r/stocks/wiki/index)

* [Pattern day trading](https://www.reddit.com/r/stocks/wiki/pdtrules)

* [Earnings calendar](https://finance.yahoo.com/calendar/earnings/)

##Karma requirements

[Click here to find how many days old your account needs to be and how much karma you need](https://www.reddit.com/r/stocks/wiki/karma) before you can comment or post to r/Stocks.

##Rules [(in depth rules wiki here)](https://www.reddit.com/r/stocks/wiki/rules)

1. Disclose any related open positions when discussing a particular stock or financial instrument.

2. Spam, ads, solicitations (including referral links), and self-promotion posts or comments will be removed and you m

In [12]:
# Importing necessary libraries
import pandas as pd
import time

# Define lists to store data
data = []

# Scraping posts & Comments
for post in subreddit.top(limit= 10): 
    data.append({
        'Type': 'Post',
        'Post_id': post.id,
        'Title': post.title,
        'Author': post.author.name if post.author else 'Unknown',
        'Timestamp': post.created_utc,
        'Text': post.selftext,
        'Score': post.score,
        'Total_comments': post.num_comments,
        'Post_URL': post.url
    })
    time.sleep(2)  # Pause for 2 second between requests

# Check if the post has comments
    if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit= 5)
        for comment in post.comments.list():
            data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0, #Comments don't have this attribute
                'Post_URL': None  #Comments don't have this attribute
            })

# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)

In [13]:
df

,Type,Post_id,Title,Author,Timestamp,Text,Score,Total_comments,Post_URL
0,Post,la34bh,"It's fucking awful seeing the ""Silver"" misinfo...",Unknown,1612186360.0,"⚠️⚠️⚠️ ***DON'T BUY SILVER, IT'S A TRAP***⚠️⚠️...",102320,4340,https://www.reddit.com/r/stocks/comments/la34b...
1,Comment,la34bh,"It's fucking awful seeing the ""Silver"" misinfo...",provoko,2021-02-01 14:58:37,"~~Just a small correction~~, FYI you can short...",1,0,None
2,Comment,la34bh,"It's fucking awful seeing the ""Silver"" misinfo...",SmithRune735,2021-02-01 13:45:17,So they went from saying redditors are bad for...,3934,0,None
3,Comment,la34bh,"It's fucking awful seeing the ""Silver"" misinfo...",Unknown,2021-02-01 13:40:32,Many of the misinformation lies on the fact th...,1472,0,None
4,Comment,la34bh,"It's fucking awful seeing the ""Silver"" misinfo...",stop-spending-money,2021-02-01 13:46:31,Owning a physical silver bar would be pretty c...,521,0,None
...,...,...,...,...,...,...,...,...,...
8342,Comment,lazcak,What $GME has taught me in 36 hours of day tra...,Axion132,2021-02-03 01:38:04,It's hard to say the play is over when there a...,5,0,None
8343,Comment,lazcak,What $GME has taught me in 36 hours of day tra...,Theta_God,2021-02-02 23:18:07,Agreed. The only thing left is a major catalys...,5,0,None
8344,Comment,lazcak,What $GME has taught me in 36 hours of day tra...,Unknown,2021-02-02 21:17:37,[deleted],8,0,None
8345,Comment,lazcak,What $GME has taught me in 36 hours of day tra...,Ikea_Man,2021-02-02 22:33:18,">I don't gamble and am very risk averse, but I...",6,0,None
